# <p style="background-color:lime; font-family:calibri; color:maroon; font-size:150%; text-align:center; border-radius:15px 50px;">1. Business Understanding</p>

# <p style="background-color:lime; font-family:calibri; color:maroon; font-size:150%; text-align:center; border-radius:15px 50px;">2. Data Understanding</p>

# <p style="background-color:#8502d1; font-family:calibri; color:white; font-size:150%; text-align:center; border-radius:15px 50px;">Step 1 | Install Libraries</p>

In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn
# !pip install matplotlib
# !pip install seaborn

# <p style="background-color:#8502d1; font-family:calibri; color:white; font-size:150%; text-align:center; border-radius:15px 50px;">Step 2 | Import Libraries</p>

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

%matplotlib inline


# <p style="background-color:#8502d1; font-family:calibri; color:white; font-size:150%; text-align:center; border-radius:15px 50px;">Step 3 | Load Dataset</p>

In [ ]:
df = pd.read_csv('penjualan.csv')

# <p style="background-color:#8502d1; font-family:calibri; color:white; font-size:150%; text-align:center; border-radius:15px 50px;">Step 4 | Data Exploration</p>

In [ ]:
# Menampilkan semua baris dan kolom
df

In [ ]:
# Menampilkan baris pertama
df.head()

In [ ]:
# Menampilkan baris terakhir
df.tail()

In [ ]:
# Menampilkan ringkasan dataframe
df.info()

In [ ]:
# Menampilkan statistik deskriptif
df.describe()

# <p style="background-color:lime; font-family:calibri; color:maroon; font-size:150%; text-align:center; border-radius:15px 50px;">3. Data Preparation</p>

In [ ]:
# 1. Mengecek nilai yang hilang
df.isnull().sum()

In [ ]:
# 2. Menghapus baris yang memiliki nilai yang hilang di kolom 'statecode'
df.dropna(subset=['statecode'], inplace=True)

In [ ]:
df.info()

In [ ]:
# 3. Mengubah tipe data kolom 'date_str' dari object ke datetime
df['orderdate'] = pd.to_datetime(df['orderdate'])
df['birthday'] = pd.to_datetime(df['birthday'])
df['deliverydate'] = pd.to_datetime(df['deliverydate'])
df['opendate'] = pd.to_datetime(df['opendate'])

In [ ]:
# 4. Mengisi nilai yang hilang untuk kolom 'deliverydate' dengan 'orderdate' + 12
# Mengisi nilai yang hilang pada kolom 'deliverydate' menggunakan np.where:
# - Jika deliverydate kosong (isna), maka diisi dengan orderdate + 12 hari
# - Jika deliverydate sudah ada isinya, maka tetap menggunakan nilai yang ada
df['deliverydate'] = np.where(df['deliverydate'].isna(),                    # Kondisi: cek apakah deliverydate kosong
                              df['orderdate'] + pd.Timedelta(days=12),       # Nilai jika kondisi True: orderdate + 12 hari
                              df['deliverydate'])                            # Nilai jika kondisi False: tetap pakai deliverydate yang ada

In [ ]:
df

In [ ]:
# 5. Mencari dan menghapus duplikat
df.drop_duplicates(inplace=True)

In [ ]:
df.info()

In [ ]:
# 6. Membuat kolom baru dengan nama profit yang menghitung selisih antara unit_price_usd dan unit_cost_usd
# Langkah 1: Mengubah tipe data menjadi float
# Mengubah kolom unit_price_usd:
# 1. Menghapus simbol $ menggunakan str.replace('$', '')
# 2. Menghapus tanda koma menggunakan str.replace(',', '') 
# 3. Mengubah string menjadi numerik menggunakan pd.to_numeric()
# 4. Parameter errors='coerce' akan mengubah nilai yang tidak valid menjadi NaN
df['unit_price_usd'] = pd.to_numeric(df['unit_price_usd'].str.replace('$', '').str.replace(',', ''), errors='coerce')

# Mengubah kolom unit_cost_usd dengan cara yang sama:
# 1. Menghapus simbol $ menggunakan str.replace('$', '')
# 2. Menghapus tanda koma menggunakan str.replace(',', '')
# 3. Mengubah string menjadi numerik menggunakan pd.to_numeric() 
# 4. Parameter errors='coerce' akan mengubah nilai yang tidak valid menjadi NaN
df['unit_cost_usd'] = pd.to_numeric(df['unit_cost_usd'].str.replace('$', '').str.replace(',', ''), errors='coerce')


In [ ]:
# Langkah 2: Buat kolom 'profit'
df['profit'] = df['unit_price_usd'] - df['unit_cost_usd']

df

In [ ]:
# Langkah 3: Mengatur ulang / indexing kolom profit menjadi disebelah unit_cost_usd
# Mendapatkan daftar semua nama kolom
columns = list(df.columns)

# Menentukan posisi kolom 'unit_price_usd' dalam daftar
position = columns.index('unit_price_usd')

# Menyisipkan 'profit' ke posisi setelah 'unit_price_usd'
# Menggunakan position+1 karena ingin menempatkannya setelah 'unit_price_usd'
columns.insert(position + 1, columns.pop(columns.index('profit')))


# Mengatur ulang kolom DataFrame berdasarkan daftar kolom yang telah disusun ulang
df = df[columns]

df

In [ ]:
# 7. Membuat kolom baru dengan nama age yang menghitung umur berdasarkan tanggal lahir
# # Pastikan kolom 'birthday' dalam format datetime
# Membuat kolom baru 'age' untuk menghitung umur
# 1. pd.Timestamp.now() mengambil waktu saat ini
# 2. Mengurangi dengan kolom 'birthday' untuk mendapatkan selisih waktu
# 3. .dt.days mengkonversi selisih waktu menjadi jumlah hari
# 4. Membagi dengan 365 (// operator pembagian bulat) untuk mendapatkan umur dalam tahun
df['age'] = (pd.Timestamp.now() - df['birthday']).dt.days // 365

In [ ]:
# Langkah 4: Mengatur ulang / indexing kolom profit menjadi disebelah unit_cost_usd
# Mendapatkan daftar semua nama kolom
columns = list(df.columns)

# Menentukan posisi kolom 'birthday' dalam daftar
position = columns.index('birthday')

# Menyisipkan 'profit' ke posisi setelah 'birthday'
# Menggunakan position+1 karena ingin menempatkannya setelah 'birthday'
columns.insert(position + 1, columns.pop(columns.index('age')))

# Mengatur ulang kolom DataFrame berdasarkan daftar kolom yang telah disusun ulang
df = df[columns]

df

In [ ]:
df=df.to_csv('penjualanBaru.csv', index=False)

In [ ]:
plt.figure(figsize=(10, 8))
corr = df.select_dtypes(include=['float64', 'int64']).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Heatmap Korelasi Variabel Numerik')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histogram untuk variabel 'age'
plt.figure(figsize=(8, 6))
sns.histplot(df['age'], bins=10, kde=True)  # kde=True menambahkan estimasi kepadatan kernel
plt.title('Distribusi Umur')
plt.xlabel('Umur')
plt.ylabel('Frekuensi')
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:







# 3. Encoding untuk variabel kategorikal
label_encoder = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])

# 4. Penanganan tipe data
# Contoh: Konversi tipe data jika diperlukan, misal konversi 'unit_price_usd' dari object ke float
# df['unit_price_usd'] = df['unit_price_usd'].str.replace('$', '').astype(float)

# 5. Penanganan outlier
# Contoh sederhana: Menghapus outlier berdasarkan Z-score untuk kolom numerik
from scipy import stats
import numpy as np
z_scores = stats.zscore(df.select_dtypes(include=[np.number]))
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df = df[filtered_entries]

# 6. Scaling fitur
scaler = MinMaxScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])

# 7. Balancing Data
# Asumsi: Target adalah kolom terakhir 'y'
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Note: Pastikan langkah-langkah di atas disesuaikan dengan struktur dan kebutuhan data Anda.

# <p style="background-color:lime; font-family:calibri; color:maroon; font-size:150%; text-align:center; border-radius:15px 50px;">4. Modelling</p>

# <p style="background-color:lime; font-family:calibri; color:maroon; font-size:150%; text-align:center; border-radius:15px 50px;">5. Evaluation</p>